<a href="https://colab.research.google.com/github/FaQ2108/Trading-Algoritmico-con-SmallCaps/blob/main/Optimizacion_Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizacion de Parametros:

## Preparacion de la Base de Datos y Entorno para Backtesting:

Se importan las librerías necesarias: pandas para manipulación de datos, numpy para cálculos numéricos, sqlite3 para interactuar con la base de datos, google.colab.drive para acceder a los archivos en Google Drive y itertools.product para generar combinaciones de parámetros.

Son definidas las siguientes constantes de cadena: DATABASE_PATH, que almacena la ruta al archivo de la base de datos SQLite; ALERT_TABLE que guarda el nombre de la tabla con la información de las alertas dentro de la base de datos. También se comenta una ruta alternativa a una base de datos similar.

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from google.colab import drive
from itertools import product

# Constantes
DATABASE_PATH = '/content/drive/MyDrive/Proyecto_SmallCaps/BBDD/bbdd_historico_pruebas/trendvision_alerts_historico.db'
#DATABASE_PATH = '/content/drive/MyDrive/Proyecto_SmallCaps/BBDD/bbdd historico actualizado/trendvision_alerts_historico.db'
ALERT_TABLE = "alerts"

## Rangos de Parametros de Optimizacion:

Se definen las variables que contienen los rangos de valores que se usarán para optimizar los parámetros de una estrategia de trading. Estos rangos son creados usando la función arange de la librería numpy.

Detalle:

Definición de Rangos de Optimización:

* RV_RANGE = np.arange(2, 4, 0.5): Se define una variable llamada RV_RANGE y se le asigna un array de numpy creado con np.arange().

np.arange(2, 4, 0.5) genera una secuencia de números que comienza en 2 (incluido), termina en 4 (excluido), y avanza con un paso de 0.5.
El resultado es un array con los valores [2.0, 2.5, 3.0, 3.5].
RV_RANGE representaría un rango de prueba para el parámetro 'rv' de una estrategia de trading.
* ONE_V_RANGE = np.arange(5000, 45000, 10000): Se define una variable llamada ONE_V_RANGE y se le asigna un array de numpy creado con np.arange().

np.arange(5000, 45000, 10000) genera una secuencia de números que comienza en 5000, termina en 45000 (excluido) y avanza con un paso de 10000.
El resultado es un array con los valores [5000, 15000, 25000, 35000].
ONE_V_RANGE representaría un rango de prueba para el parámetro 'one_v' de una estrategia de trading.
* CHANGE_RANGE = np.arange(0, 10, 2): Se define una variable llamada CHANGE_RANGE y se le asigna un array de numpy creado con np.arange().

np.arange(0, 10, 2) genera una secuencia de números que comienza en 0, termina en 10 (excluido) y avanza con un paso de 2.
El resultado es un array con los valores [0, 2, 4, 6, 8].
CHANGE_RANGE representaría un rango de prueba para el parámetro 'change' de una estrategia de trading.
* PRICE_RANGE = np.arange(5, 50, 5): Se define una variable llamada PRICE_RANGE y se le asigna un array de numpy creado con np.arange().

np.arange(5, 50, 5) genera una secuencia de números que comienza en 5, termina en 50 (excluido) y avanza con un paso de 5.
El resultado es un array con los valores [5, 10, 15, 20, 25, 30, 35, 40, 45].
PRICE_RANGE representaría un rango de prueba para el parámetro 'price' de una estrategia de trading.
* FT_RANGE = np.arange(0.5, 5.5, 1): Se define una variable llamada FT_RANGE y se le asigna un array de numpy creado con np.arange().

np.arange(0.5, 5.5, 1) genera una secuencia de números que comienza en 0.5, termina en 5.5 (excluido) y avanza con un paso de 1.
El resultado es un array con los valores [0.5, 1.5, 2.5, 3.5, 4.5].
FT_RANGE representaría un rango de prueba para el parámetro 'ft' de una estrategia de trading.

In [ ]:
# RANGOS DE OPTIMIZACION
RV_RANGE = np.arange(2, 4, 0.5)
ONE_V_RANGE = np.arange(5000, 45000, 10000)
CHANGE_RANGE = np.arange(0, 10, 2)
PRICE_RANGE = np.arange(5, 50, 5)
FT_RANGE = np.arange(0.5, 5.5, 1)

def connect_to_database(file_path):
    """Establece una conexión a la base de datos SQLite."""
    try:
        drive.mount('/content/drive')
        conexion = sqlite3.connect(file_path)
        return conexion
    except sqlite3.Error as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

In [ ]:
def load_alerts_from_db(conexion, table_name):
    """Carga las alertas desde la tabla especificada de la base de datos."""
    alerts_query = f"SELECT * FROM {table_name}"
    alerts = pd.read_sql_query(alerts_query, conexion)
    return alerts

In [ ]:
def filter_alerts(alerts_df):
    """Filtra las alertas según las condiciones."""
    filtered_alerts = alerts_df[
        (alerts_df['ohlc_downloaded'] == True) &
        (alerts_df['ranking'] == '#1') &
        (alerts_df['rv'] > 1) &
        (alerts_df['one_v'] > 1000) &
        (pd.to_datetime(alerts_df['date'], format="%Y-%m-%d %H:%M:%S").dt.time >= pd.to_datetime("15:30:00").time()) &
        (pd.to_datetime(alerts_df['date'], format="%Y-%m-%d %H:%M:%S").dt.time <= pd.to_datetime("22:00:00").time()) &
        (alerts_df['change'] > 0) &
        (alerts_df['price'] >= 0) & (alerts_df['price'] <= 20) &
        (alerts_df['ft'] > 1) &
        (alerts_df['mc'] < 100000000000000) &
        (alerts_df['rv'] < 20) &
        (alerts_df['mv'] > 10)
    ].copy()

    print(f"Total de alertas válidas: {len(filtered_alerts)}")
    return filtered_alerts

In [ ]:
def generate_param_combinations(rv_range, one_v_range, change_range, price_range, ft_range):
    """Genera todas las combinaciones posibles de parámetros para la optimización."""
    param_combinations = pd.DataFrame(list(product(rv_range, one_v_range, change_range, price_range, ft_range)),
                                       columns=['rv_threshold', 'one_v_threshold', 'change_threshold',
                                                'price_threshold', 'ft_threshold'])
    return param_combinations

In [ ]:
def fetch_ohlc_data(conexion, ticker, alert_date):
    """Obtiene los datos OHLC para un ticker y fecha específicos."""
    ohlc_query = f"""
        SELECT * FROM ohlc
        WHERE ticker = '{ticker}'
          AND DATE(date) = DATE('{alert_date.date()}')
          AND date >= '{alert_date}'
    """
    ohlc_data = pd.read_sql_query(ohlc_query, conexion)
    return ohlc_data

In [ ]:
def calculate_exit_price(ohlc_data, entry_price):
    """Calcula el precio de salida según la estrategia."""
    # Calcular máximo desde el punto de entrada
    ohlc_data['high_since_entry'] = ohlc_data['high'].cummax()

    # Inicializar valores de Stop Loss y condiciones
    stop_loss = entry_price * 0.91  # 9% por debajo del precio de entrada
    stop_loss_adjusted = entry_price * 1.13  # Stop ajustado al 13% por encima si sube más de un 20%
    take_profit = ohlc_data['high_since_entry'].max()  # El máximo del día como Take Profit
    exit_price = None
    exit_time = None

    # Variables para detectar techo
    peak_price = entry_price
    consecutive_lower_highs = 2
    ceiling_threshold = 30  # Número de velas consecutivas con máximos más bajos para considerar un techo

    # Recorrer los datos para determinar salida
    for _, row in ohlc_data.iterrows():
        # Ajustar Stop Loss si el precio sube un 20% o más
        if row['high_since_entry'] >= entry_price * 1.13:
            stop_loss = stop_loss_adjusted

        # Verificar si se activa el Stop Loss
        if row['low'] <= stop_loss:
            exit_price = stop_loss
            exit_time = row['date']
            break

        # Verificar si se alcanza el Take Profit
        if row['high'] >= take_profit:
            exit_price = take_profit
            exit_time = row['date']
            break

        # Detectar techo
        if row['high'] > peak_price:
            peak_price = row['high']
            consecutive_lower_highs = 0
        elif row['high'] < peak_price:
            consecutive_lower_highs += 1
        else:
            consecutive_lower_highs = 0

        # Salir si se detecta un techo
        if consecutive_lower_highs >= ceiling_threshold:
            exit_price = row['close']
            exit_time = row['date']
            break

    # Si no se alcanzó ni el Stop Loss, ni el Take Profit, ni se detectó un techo, tomar el máximo del día como Take Profit
    if exit_price is None:
        exit_price = take_profit
        exit_time = ohlc_data.loc[ohlc_data['high_since_entry'].idxmax(), 'date']

    return exit_price, exit_time

In [ ]:
def run_backtest(filtered_alerts, conexion, rv_threshold, one_v_threshold,
                 change_threshold, price_threshold, ft_threshold):
    """Ejecuta el backtest de la estrategia de trading."""
    results = []
    for _, alert in filtered_alerts.iterrows():
        ticker = alert['ticker']
        alert_date = pd.to_datetime(alert['date'])
        entry_price = alert['price']

        # Obtener datos OHLC
        ohlc_data = fetch_ohlc_data(conexion, ticker, alert_date)
        if ohlc_data.empty:
            continue

        # Calcular precio y tiempo de salida
        exit_price, exit_time = calculate_exit_price(ohlc_data, entry_price)

        # Calcular rendimiento
        pnl_percent = ((exit_price - entry_price) / entry_price) * 100

        # Guardar resultado
        results.append(pnl_percent)

    return results

In [ ]:
def optimize_parameters(filtered_alerts, conexion, param_combinations):
    """Optimiza los parámetros de la estrategia."""
    optimization_results = []
    for _, params in param_combinations.iterrows():
        rv_threshold = params['rv_threshold']
        one_v_threshold = params['one_v_threshold']
        change_threshold = params['change_threshold']
        price_threshold = params['price_threshold']
        ft_threshold = params['ft_threshold']

        results = run_backtest(filtered_alerts, conexion, rv_threshold, one_v_threshold,
                              change_threshold, price_threshold, ft_threshold)
        avg_pnl = np.mean(results) if results else 0
        optimization_results.append({
            'rv_threshold': rv_threshold,
            'one_v_threshold': one_v_threshold,
            'change_threshold': change_threshold,
            'price_threshold': price_threshold,
            'ft_threshold': ft_threshold,
            'average_pnl': avg_pnl
        })

    return pd.DataFrame(optimization_results)

In [ ]:
def main():
    """Función principal para ejecutar el script de optimización."""
    conexion = connect_to_database(DATABASE_PATH)
    if conexion:
        alerts_df = load_alerts_from_db(conexion, ALERT_TABLE)
        filtered_alerts = filter_alerts(alerts_df)
        param_combinations = generate_param_combinations(RV_RANGE, ONE_V_RANGE, CHANGE_RANGE,
                                                        PRICE_RANGE, FT_RANGE)

        optimization_results_df = optimize_parameters(filtered_alerts, conexion, param_combinations)
        print(optimization_results_df)

        conexion.close()


if __name__ == "__main__":
    main()

Mounted at /content/drive
Total de alertas válidas: 60
      rv_threshold  one_v_threshold  change_threshold  price_threshold  \
0              2.0           5000.0               0.0              5.0   
1              2.0           5000.0               0.0              5.0   
2              2.0           5000.0               0.0              5.0   
3              2.0           5000.0               0.0              5.0   
4              2.0           5000.0               0.0              5.0   
...            ...              ...               ...              ...   
3595           3.5          35000.0               8.0             45.0   
3596           3.5          35000.0               8.0             45.0   
3597           3.5          35000.0               8.0             45.0   
3598           3.5          35000.0               8.0             45.0   
3599           3.5          35000.0               8.0             45.0   

      ft_threshold  average_pnl  
0              0.5    